In [4]:
#Download the docs
!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json

--2024-06-18 21:05:40--  https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json [following]
--2024-06-18 21:05:41--  https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.1’

documents.json.1    100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-06-18 21:05:41 (46.6 MB/s) - ‘documents.json.1’ saved [65

In [5]:
!head documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {


In [1]:
import json

with open('./documents.json', 'rt') as f:
    documents_file = json.load(f)

documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
len(documents)

948

In [10]:
#Push this to our database

#in ES we have Indexes analogous to table in RDB

In [97]:
#Connect to elastic search
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7a8f22762ec0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [81]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7a8f2260cca0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [16]:
##ADDING documents to Elastic Search

from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

/home/codespace/.local/share/virtualenvs/llm-rag-J7PTIVi-/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 41.76it/s]


In [17]:
user_question = "How do I join the course after it has started?"

#ES cannot accommadte direct search of text if the words differ like instead of started if they use begun it cannot accommodate direct

#We can create vectors to looki for similarities and retrieve similar question and give output

In [18]:
# ^3 indicates that if user question matches the question in doc, it is given more priority when it matches with text or section
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text", "section"], 
                    "type": "best_fields"
                }
            },
            "filter": { 
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [19]:
response = es.search(index=index_name, body=search_query)

In [20]:
response

ObjectApiResponse({'took': 996, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 407, 'relation': 'eq'}, 'max_score': 70.93361, 'hits': [{'_index': 'course-questions', '_id': 'if7fUpABu1U5aAdqvmJb', '_score': 70.93361, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'jv7fUpABu1U5aAdqvmLP', '_score': 62.66832, '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next co

In [9]:
for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")
    

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess

In [14]:
#Use this response and add it to prompt so that llm can generate responses based on this info.

In [21]:
es = Elasticsearch("http://localhost:9200")

def retrieve_documents(query, index_name="course-questions", max_results=5):
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [33]:
user_question = "How do I execute a command in a running docker container?"

response = retrieve_documents(user_question, boost=4)
print(type(response), len(response), response[0].keys())

for doc in response:
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

<class 'list'> 5 dict_keys(['text', 'section', 'question', 'course'])
Section: Module 1: Docker and Terraform
Question: PGCLI - running in a Docker container
Answer: In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.
Below the usage with values used in the videos of the course for:
network name (docker network)
postgres related variables for pgcli
Hostname
Username
Port
Database name
$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1
175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi
Password for root:
Server: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)
Version: 4.0.1
Home: http://pgcli.com
root@pg-database:ny_taxi> \dt
+--------+------------------+-------+-------+
| Schema | Name             | Type  | Owner |
|--------+------------------+-------+-------|
| public | yellow_taxi_data | table | root  |
+--------+------------------+-------+-------+
SELECT 1
Time: 0.009s

In [14]:
##Connect ChatGPT

from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": "What's the formula for Energy?"}]
)
print(response.choices[0].message.content)

The formula for energy is E = mc^2, where E represents energy, m represents mass, and c represents the speed of light in a vacuum.


In [15]:
context_docs = retrieve_documents(user_question)

context = ""

for doc in context_docs:
    doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    context += doc_str

context = context.strip()
print(context)

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess y

In [24]:
prompt = f"""
You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contan the answer, return "NONE"

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

In [25]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}]
)
answer = response.choices[0].message.content
answer

'You can still join the course after it has started, but be aware of deadlines for final projects.'

In [26]:
prompt = f"""
QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

In [27]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{
            "role": "system", "content": """You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contan the answer, return 'NONE'"""
        , "role": "user", "content": prompt}]
)
answer = response.choices[0].message.content
answer


"If you have missed the start date of the course, you can still join and follow along. However, make sure to catch up on any missed material and be aware of upcoming deadlines for assignments and projects. It's always a good idea to stay proactive and not leave everything until the last minute. If the course has already finished, don't worry as you can still access all the materials and continue learning at your own pace. Additionally, you can work on any unfinished assignments or projects to further enhance your skills. To prepare before the course starts, make sure to install all required software and familiarize yourself with the prerequisites and syllabus. This way, you'll be ready to hit the ground running once the course begins."

In [18]:
def build_context(documents):
    context = ""

    for doc in documents:
        doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
        context += doc_str
    
    context = context.strip()
    return context


def build_prompt(user_question, documents):
    context = build_context(documents)
    return f"""
You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

def ask_openai(prompt, model="gpt-3.5-turbo-0125"):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer
def qa_bot(user_question):
    context_docs = retrieve_documents(user_question)
    prompt = build_prompt(user_question, context_docs)
    answer = ask_openai(prompt)
    return answer

In [28]:
qa_bot("I'm getting invalid reference format: repository name must be lowercase")

'\nWith: `port: \'{{ env_var("POSTGRES_PORT", 5432) }}\'`'

In [29]:
qa_bot("I can't connect to postgres port 5432, my password doesn't work")

'Based on the provided context, the issue you are facing with the connection to postgres port 5432 and password authentication failed for user "root" may be due to the port being taken by another postgres or the role "root" not existing. \n\nTo resolve this, try substituting port 5432 with a different port, checking if there is a service in Windows running postgres, or changing the port from 5432 to resolve the issue. Additionally, ensure that the required role exists for login and consider changing the PostgreSQL user configuration settings as mentioned in the FAQ answers.'

In [2]:
#HOMEWORK

In [3]:
#Question 1
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
!pip install requests

In [ ]:
# Question 2
##ADDING documents to Elastic Search

from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

In [34]:
#Question 3

user_question = "How do I execute a command in a running docker container?"
es = Elasticsearch("http://localhost:9200")

# ^3 indicates that if user question matches the question in doc, it is given more priority when it matches with text or section
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text", "section"], 
                    "type": "best_fields"
                }
            },
            "filter": { 
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [35]:
response = es.search(index=index_name, body=search_query)


In [41]:
response['hits'].keys()

dict_keys(['total', 'max_score', 'hits'])

In [43]:
response['hits']

{'total': {'value': 407, 'relation': 'eq'},
 'max_score': 75.54128,
 'hits': [{'_index': 'course-questions',
   '_id': '8f7fUpABu1U5aAdqyGKa',
   '_score': 75.54128,
   '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+-------------

In [44]:
documents = [hit['_source'] for hit in response['hits']['hits']]
documents

[{'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
  'section': 'Module 1: Docker and Terraform',
  'question': 'PGCLI - running in a Docker container',
 

In [80]:
#Question 4

#Question 3

user_question = "How do I execute a command in a running docker container?"

# ^3 indicates that if user question matches the question in doc, it is given more priority when it matches with text or section
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text", "section"], 
                    "type": "best_fields"
                }
            },
            "filter": { 
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es.search(index=index_name, body=search_query)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7a8f22b14ee0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [73]:
documents = [hit['_source'] for hit in response['hits']['hits']]
for doc in documents:
    print(doc['question'])

TypeError: 'ChatCompletion' object is not subscriptable

In [51]:
#Question 5
context_template = """
Q: {question}
A: {text}
""".strip()

In [52]:

context = ""

for doc in documents:
    doc_str = f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    context += doc_str

context = context.strip()
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [99]:
user_question = "How do I execute a command in a running docker container?"

prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {user_question}

CONTEXT:
{context}
""".strip()

In [100]:
len(prompt_template)

1462

In [104]:
#Question 6
# !pip install tiktoken
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0125")

In [105]:
encoded_tokens = encoding.encode(prompt_template)

In [106]:
len(encoded_tokens)

325

In [62]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt_template}]
)
answer = response.choices[0].message.content
answer

'To execute a command in a running docker container, you can use the following command:\ndocker exec -it <container-id> bash'